In [19]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name



In [ ]:
# Build Docker image and push to ECR
import subprocess

agent_name = "deep_research_scoping_agent"

# Get AWS account ID
account_id = boto_session.client("sts").get_caller_identity()["Account"]

# ECR repo URI
ecr_repo = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{agent_name}"

# Create ECR repository if it doesn't exist
print(f"Creating ECR repository: {agent_name}...")
ecr_client = boto_session.client("ecr")
try:
    ecr_client.create_repository(
        repositoryName=agent_name,
        imageScanningConfiguration={'scanOnPush': True}
    )
    print(f"✅ ECR repository '{agent_name}' created successfully")
except ecr_client.exceptions.RepositoryAlreadyExistsException:
    print(f"ℹ️  ECR repository '{agent_name}' already exists")

# Authenticate Docker to ECR
print("Authenticating to ECR...")
login_password = subprocess.run(
    ["aws", "ecr", "get-login-password", "--region", region],
    capture_output=True,
    text=True,
    check=True
).stdout

subprocess.run(
    ["docker", "login", "--username", "AWS", "--password-stdin", f"{account_id}.dkr.ecr.{region}.amazonaws.com"],
    input=login_password,
    text=True,
    check=True
)

# Build Docker image (run from parent directory)
print(f"Building Docker image: {agent_name}...")
subprocess.run(
    ["docker", "build", "-f", "src/Dockerfile", "-t", agent_name, "."],
    cwd="/workspaces/agent_core_deep_research",
    check=True
)

# Tag Docker image
print(f"Tagging image for ECR: {ecr_repo}:latest...")
subprocess.run(
    ["docker", "tag", f"{agent_name}:latest", f"{ecr_repo}:latest"],
    check=True
)

# Push Docker image to ECR
print(f"Pushing image to ECR: {ecr_repo}:latest...")
subprocess.run(
    ["docker", "push", f"{ecr_repo}:latest"],
    check=True
)

print(f"✅ Docker image successfully pushed to {ecr_repo}:latest")

Creating ECR repository: deep_research_scoping_agent...
ℹ️  ECR repository 'deep_research_scoping_agent' already exists
Authenticating to ECR...
ℹ️  ECR repository 'deep_research_scoping_agent' already exists
Authenticating to ECR...
Login Succeeded
Building Docker image: deep_research_scoping_agent...
Login Succeeded
Building Docker image: deep_research_scoping_agent...


ERROR: failed to build: resolve : lstat src: no such file or directory


CalledProcessError: Command '['docker', 'build', '-f', 'src/Dockerfile', '-t', 'deep_research_scoping_agent', '.']' returned non-zero exit status 1.

In [14]:
# Create CodeBuild execution role
import json

iam_client = boto_session.client("iam")
role_name = f"deep-research-codebuild-role-{region}"

# IAM trust policy for CodeBuild
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "codebuild.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# IAM inline policy for CodeBuild
codebuild_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "CloudWatchLogs",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": f"arn:aws:logs:{region}:{account_id}:log-group:/aws/codebuild/*"
        },
        {
            "Sid": "ECRAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:BatchCheckLayerAvailability",
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchGetImage",
                "ecr:GetAuthorizationToken",
                "ecr:PutImage",
                "ecr:InitiateLayerUpload",
                "ecr:UploadLayerPart",
                "ecr:CompleteLayerUpload"
            ],
            "Resource": [
                f"arn:aws:ecr:{region}:{account_id}:repository/{agent_name}",
                "*"
            ]
        },
        {
            "Sid": "S3SourceAccess",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:GetObjectVersion"
            ],
            "Resource": "arn:aws:s3:::*/*"
        }
    ]
}

# Create role
try:
    print(f"Creating IAM role: {role_name}...")
    create_role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="CodeBuild execution role for Bedrock AgentCore"
    )
    role_arn = create_role_response["Role"]["Arn"]
    print(f"✅ IAM role created: {role_arn}")
    
    # Attach inline policy
    print("Attaching inline policy...")
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName="CodeBuildPolicy",
        PolicyDocument=json.dumps(codebuild_policy)
    )
    print("✅ Inline policy attached successfully")
    
except iam_client.exceptions.EntityAlreadyExistsException:
    print(f"ℹ️  IAM role '{role_name}' already exists")
    role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"
    
    # Update inline policy if role exists
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName="CodeBuildPolicy",
            PolicyDocument=json.dumps(codebuild_policy)
        )
        print("✅ Inline policy updated successfully")
    except Exception as e:
        print(f"⚠️  Warning: Could not update policy: {e}")

print(f"\nCodeBuild Role ARN: {role_arn}")

Creating IAM role: deep-research-codebuild-role-us-west-2...
ℹ️  IAM role 'deep-research-codebuild-role-us-west-2' already exists
✅ Inline policy updated successfully

CodeBuild Role ARN: arn:aws:iam::482387069690:role/deep-research-codebuild-role-us-west-2
ℹ️  IAM role 'deep-research-codebuild-role-us-west-2' already exists
✅ Inline policy updated successfully

CodeBuild Role ARN: arn:aws:iam::482387069690:role/deep-research-codebuild-role-us-west-2


In [15]:
agentcore_runtime = Runtime()

agent_name = "deep_research_scoping_agent"

# Get AWS account ID and construct ECR image URI
account_id = boto_session.client("sts").get_caller_identity()["Account"]
ecr_repository = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{agent_name}"

response = agentcore_runtime.configure(
    entrypoint="agent_core/scoping_agent_core.py",
    auto_create_execution_role=True,
    code_build_execution_role=role_arn,  # Use the role we created
    auto_create_ecr=False,  
    region=region,
    agent_name=agent_name,
    ecr_repository=ecr_repository  # Use the ECR image we built and pushed
)
response

Entrypoint parsed: file=/workspaces/agent_core_deep_research/src/agent_core/scoping_agent_core.py, bedrock_agentcore_name=scoping_agent_core
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: deep_research_scoping_agent
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: deep_research_scoping_agent
Memory disabled
Network mode: PUBLIC
Generated Dockerfile: Dockerfile
Generated .dockerignore: /workspaces/agent_core_deep_research/src/.dockerignore
Memory disabled
Network mode: PUBLIC
Generated Dockerfile: Dockerfile
Generated .dockerignore: /workspaces/agent_core_deep_research/src/.dockerignore
Keeping 'deep_research_scoping_agent' as default agent
Bedrock AgentCore configured: /workspaces/agent_core_deep_research/src/.bedrock_agentcore.yaml
Keeping 'deep_research_scoping_agent' as default agent
Bedrock AgentCore configured: /workspaces/agent_core_deep_research/src/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/workspaces/agent_core_deep_research/src/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/workspaces/agent_core_deep_research/src/Dockerfile'), dockerignore_path=PosixPath('/workspaces/agent_core_deep_research/src/.dockerignore'), runtime='Docker', runtime_type=None, region='us-west-2', account_id='482387069690', execution_role=None, ecr_repository='482387069690.dkr.ecr.us-west-2.amazonaws.com/deep_research_scoping_agent', auto_create_ecr=False, s3_path=None, auto_create_s3=False, memory_id=None, network_mode='PUBLIC', network_subnets=None, network_security_groups=None, network_vpc_id=None)

In [16]:
launch_result = agentcore_runtime.launch()

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'deep_research_scoping_agent' to account 482387069690 (us-west-2)
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'deep_research_scoping_agent' to account 482387069690 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Using ECR repository from config: 482387069690.dkr.ecr.us-

In [17]:
payload = {
    "prompt": "I want to research the best coffee shops in San Francisco.",
    "thread_id": "test-thread-1"
}
result = agentcore_runtime.invoke(payload)




RuntimeClientError: An error occurred (RuntimeClientError) when calling the InvokeAgentRuntime operation: An error occurred when starting the runtime. Please check your CloudWatch logs for more information.